<a href="https://colab.research.google.com/github/PranjalKBajaria/contra-infodemic/blob/main/COVINFOnotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install transformers

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
real_data = pd.read_csv("all_real.csv")

In [ ]:
real_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5691 entries, 0 to 5690
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   coordinates                 34 non-null     object 
 1   created_at                  5691 non-null   object 
 2   hashtags                    2460 non-null   object 
 3   media                       2505 non-null   object 
 4   urls                        2396 non-null   object 
 5   favorite_count              5691 non-null   int64  
 6   id                          5691 non-null   int64  
 7   in_reply_to_screen_name     1354 non-null   object 
 8   in_reply_to_status_id       1271 non-null   float64
 9   in_reply_to_user_id         1354 non-null   float64
 10  lang                        5691 non-null   object 
 11  place                       207 non-null    object 
 12  possibly_sensitive          4144 non-null   object 
 13  retweet_count               5691 

In [ ]:
real_data = real_data[['text']]

In [ ]:
real_data

,text,label
0,What is a coronavirus immunity certificate? US...,0.0
1,@oluafolabi What is a Coronavirus in the prese...,0.0
2,What is a coronavirus immunity certificate? US...,0.0
3,What is a coronavirus immunity certificate? US...,0.0
4,@AngryDuck91 - Does WHO recommend that all int...,0.0
...,...,...
5686,Can people wear masks while exercising during ...,0.0
5687,Can people wear masks while Exercising? \n\nPe...,0.0
5688,Q: Can people wear masks while exercising? \nA...,0.0
5689,Can people wear masks while exercising during ...,0.0


In [ ]:
real_data['label'] = [np.float(0)]*5691

In [ ]:
real_data

,text,label
0,What is a coronavirus immunity certificate? US...,0.0
1,@oluafolabi What is a Coronavirus in the prese...,0.0
2,What is a coronavirus immunity certificate? US...,0.0
3,What is a coronavirus immunity certificate? US...,0.0
4,@AngryDuck91 - Does WHO recommend that all int...,0.0
...,...,...
5686,Can people wear masks while exercising during ...,0.0
5687,Can people wear masks while Exercising? \n\nPe...,0.0
5688,Q: Can people wear masks while exercising? \nA...,0.0
5689,Can people wear masks while exercising during ...,0.0


In [ ]:
fake_data = pd.read_csv('fake_dataset_comb.csv')

In [ ]:
fake_data = fake_data[['text']]

In [ ]:
real_data = real_data[:4500]

In [ ]:
fake_data['label'] = [np.float(1)]*4261 

In [ ]:
full_dataset = pd.concat([real_data, fake_data])

In [ ]:
full_dataset

,text,label
0,What is a coronavirus immunity certificate? US...,0.0
1,@oluafolabi What is a Coronavirus in the prese...,0.0
2,What is a coronavirus immunity certificate? US...,0.0
3,What is a coronavirus immunity certificate? US...,0.0
4,@AngryDuck91 - Does WHO recommend that all int...,0.0
...,...,...
4256,Face masks don’t “protect against covid19 and ...,1.0
4257,Antibodies for the common cold produce a posit...,1.0
4258,Ghislaine Maxwell — the recently arrested asso...,1.0
4259,U.S. hospital laboratories falsify COVID-19 te...,1.0


In [ ]:
full_dataset = full_dataset.sample(frac=1, random_state=101)

In [ ]:
clean_df = full_dataset

#removing links
clean_df['text'] = clean_df['text'].str.replace(r'http(\S)+', r'')

#replacing the html codes
clean_df['text'] = clean_df['text'].str.replace(r'&amp;?',r'and')
clean_df['text'] = clean_df['text'].str.replace(r'&lt;',r'<')
clean_df['text'] = clean_df['text'].str.replace(r'&gt;',r'>')

#getting rid of the 'RT', any '@account_name' and everything that is not a letter
clean_df['text'] = clean_df['text'].str.lower().str.replace('rt|@[^\s]+|[^A-Za-z ]', '', regex=True)

In [ ]:
full_dataset = clean_df

In [ ]:
full_dataset.reset_index(inplace=True)
full_dataset.drop(columns=['index'], axis=1, inplace=True)
full_dataset

,text,label
0,what are the symptoms of covid these are the ...,0.0
1,it doesn t contain hiv but it has protein mak...,1.0
2,democrat michigan governor gretchen whitmer b...,1.0
3,what is a coronaviruscoronaviruses are a large...,0.0
4,why do we need it its based on what facts yes...,0.0
...,...,...
8756,people died last year in the us in weeks ...,0.0
8757,belgium health minister puts ban on non essen...,1.0
8758,says theres an number you can call to track t...,1.0
8759,how does covid spread odisha,0.0


In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

,text,label
0,What is a coronavirus immunity certificate? US...,0
1,@oluafolabi What is a Coronavirus in the prese...,0
2,What is a coronavirus immunity certificate? US...,0
3,What is a coronavirus immunity certificate? US...,0
4,@AngryDuck91 - Does WHO recommend that all int...,0
...,...,...
4495,Taking a hot bath does not prevent the new cor...,0
4496,#COVIDăź19 Can cooking foods kill any potenti...,0
4497,FACT: Taking a hot bath does not prevent the n...,0
4498,FACT: Taking a hot bath does not prevent the n...,0


In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(full_dataset['text'], full_dataset['label'])

In [ ]:
train_texts = train_texts.to_list()
test_texts = test_texts.to_list()

In [ ]:
type(train_texts)

list

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
train_encodings

{'input_ids': [[101, 2522, 17258, 7865, 2064, 2022, 11860, 1999, 2752, 2007, 2980, 1998, 14178, 24734, 2013, 1996, 3350, 2061, 2521, 1996, 2522, 17258, 7865, 2064, 2022, 11860, 1999, 2035, 2752, 2164, 2752, 2007, 2980, 1998, 14178, 4633, 7539, 1997, 4785, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2003, 2045, 2505, 1045, 2323, 2025, 2079, 1996, 2206, 5761, 2024, 2025, 4621, 2114, 2522, 17258, 1998, 2064, 2022, 17631, 9422, 2635, 3151, 27849, 2128, 7583, 3111, 4147, 3674, 15806, 2635, 2969, 7583, 21261, 2107, 2004, 24479, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2903, 2009, 2030, 2025, 2023, 2003, 1999, 2739, 5706, 2740, 2704, 8509, 7221, 2006, 2512, 6827, 4424, 3450, 1997, 5381, 2030, 3618, 1999, 7169, 2752, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
#creating the dataset
class CovidDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CovidDataset(train_encodings, train_labels)
test_dataset = CovidDataset(test_encodings, test_labels)

In [ ]:
train_dataset.__getitem__(1)

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,  2003,  2045,  2505,  1045,  2323,  2025,  2079,  1996,  2206,
          5761,  2024,  2025,  4621,  2114,  2522, 17258,  1998,  2064,  2022,
         17631,  9422,  2635,  3151, 27849,  2128,  7583,  3111,  4147,  3674,
         15806,  2635,  2969,  7583, 21261,  2107,  2004, 24479,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 'labels

In [ ]:
from torch.utils.data import DataLoader
from transformers import  DistilBertModel

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

optim = torch.optim.Adam(params =  model.parameters(), lr=0.005)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = full_dataset['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
tokenized

0       [101, 2054, 2024, 1996, 8030, 1997, 2522, 1725...
1       [101, 2009, 2987, 1056, 5383, 9820, 2021, 2009...
2       [101, 7672, 4174, 3099, 21625, 1059, 16584, 50...
3       [101, 2054, 2003, 1037, 21887, 23350, 27108, 7...
4       [101, 2339, 2079, 2057, 2342, 2009, 2049, 2241...
                              ...                        
8756    [101, 2111, 2351, 2197, 2095, 1999, 1996, 2149...
8757    [101, 5706, 2740, 2704, 8509, 7221, 2006, 2512...
8758    [101, 2758, 2045, 2015, 2019, 2193, 2017, 2064...
8759     [101, 2129, 2515, 2522, 17258, 3659, 21874, 102]
8760    [101, 2054, 2003, 21887, 23350, 2129, 2515, 25...
Name: text, Length: 8761, dtype: object

In [ ]:
tokenized = clean_df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 280

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(8761, 280)

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)